# SMNA-Dashboard

Este notebook trata da organização dos resultados do GSI em relação à minimização da função custo do 3DVar. Especificamente, são tratadas as seguintes informações, obtidas a partir do arquivo de log do GSI:

```
Begin Jo table outer loop
    Observation Type           Nobs                        Jo        Jo/n
surface pressure             101588    3.6963295810242533E+04       0.364
temperature                   54009    8.4362268122117763E+04       1.562
wind                         174592    2.2575676441332555E+05       1.293
moisture                      21242    9.4984707783307513E+03       0.447
gps                          280594    5.4391758277321467E+05       1.938
radiance                     171806    1.8338082096514766E+05       1.067
                               Nobs                        Jo        Jo/n
           Jo Global         803831    1.0838792028623789E+06       1.348
End Jo table outer loop
```

A depender da quantidade de outer e inner loops, o GSI registra um número diferente de informações sobre o número de observações consideradas (`Nobs`), o custo da minimização (`Jo`) e o custo da minimização normalizado pelo número de observações (`Jo/n`). A configuração do GSI/3DVar aplicado ao SMNA (válido para a data de escrita deste notebook), considera `miter=2` e `niter=3`, ou seja, 2 outer loops com 3 inner loops cada. Nesse sentido, as informações obtidas a partir das iterações do processo de minimização da função custo, consideram o seguinte:

* OMF: início do primeiro outer loop, onde o estado do sistema é dado pelo background;
* OMF (1st INNER LOOP): final do primeiro inner loop do primeiro outer loop, onde o estado do sistema ainda é dado pelo background;
* OMF (2nd INNER LOOP): final do segundo inner loop do primeiro outer loop, onde o estado do sistema ainda é dado pelo background;
* OMA (AFTER 1st OUTER LOOP): início do segundo outer loop, onde o estado do sistema é dado pela análise;
* OMA (1st INNER LOOP): final do primeiro inner loop do segundo outer loop, onde o estado do sistema é dado pela análise;
* OMA (2nd INNER LOOP): final do segundo inner loop do segundo outer loop, onde o estado do sistema é dado pela análise;
* OMA (AFTER 2nd OUTER LOOP): final do segundo outer loop, análise final.

**Nota:** as informações das iterações `OMF` e `OMF (1st INNER LOOP)` são iguais, assim como as informações das iterações `OMA (AFTER 1st OUTER LOOP)` e `OMA (1st INNER LOOP)`.

As informações do log do GSI são organizadas em um dataframe com a marcação das datas e a inclusão das informações sobre os outer e inner loops:

```
     Date                Observation Type Nobs   Jo            Jo/n  Iter
   0 2023-02-16 06:00:00 surface pressure 104308 32537.652151  0.312 OMF
   1 2023-02-16 06:00:00 temperature      25065  9857.265337   0.393 OMF
   2 2023-02-16 06:00:00 wind             127888 61267.072233  0.479 OMF
   3 2023-02-16 06:00:00 moisture         8705   2103.832442   0.242 OMF
   4 2023-02-16 06:00:00 gps              291665 600962.196931 2.060 OMF
 ...                 ...              ...    ...           ...   ... ...
5399 2023-03-16 00:00:00 wind             203048 129312.187759 0.637 OMA (AFTER 2nd OUTER LOOP)
5400 2023-03-16 00:00:00 moisture         22219  4948.997007   0.223 OMA (AFTER 2nd OUTER LOOP)
5401 2023-03-16 00:00:00 gps              264466 392890.280946 1.486 OMA (AFTER 2nd OUTER LOOP)
5402 2023-03-16 00:00:00 radiance         183884 56169.185410  0.305 OMA (AFTER 2nd OUTER LOOP)
5403 2023-03-16 00:00:00 Jo Global        832986 645663.456547 0.775 OMA (AFTER 2nd OUTER LOOP)
```

Considerando vários experimentos, os dataframes são concatenados em um só (`dfs`), o qual é salvo em disco no formato CSV. A indexação do dataframe `dfs` pode ser feita da seguinte forma:

1. Escolha de um subdataframe: 

    `df_dtc1 = dfs.xs('df_dtc', axis=1)`

2. Escolha de uma variável: 

    `df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].reset_index(drop=True)`
    
3. Escolha de um parâmetro: 

    `df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].loc[df_dtc1['Iter'] == 'OMF'].reset_index(drop=True)`
    
4. Escolha de um horário: 

    `df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].loc[df_dtc1['Iter'] == 'OMF'].set_index('Date').at_time(str('00:00:00')).reset_index(drop=False)`

**Nota:** nesta versão, a consideração dos valores de `miter` e `niter` não está generalizada. Para outras configurações, será necessário ajustar os valores de `i` na função `df_Nobs`.

---

Carlos Frederico Bastarz (carlos.bastarz@inpe.br), Abril de 2023.

In [1]:
import os
import re
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Função para ler os arquivos de log e separar as seções que se iniciam por 'Begin Jo'
# Nota: as opções 'outer' e 'inner' esão sendo desconsideradas

def df_Nobs(fname, nexp, mname):
   
    colnames = ['Observation Type', 'Nobs', 'Jo', 'Jo/n', 'Iter']
    
    dfNobs = pd.DataFrame(columns=colnames)
    
    # definição das palavras-chave que devem constar no começo das linhas de interesse
    begin = 'Begin Jo'
    end = 'End Jo'
    
    with open(fname, 'r') as file: # abre o arquivo para leitura
        match = False
        i = 0

        for line in file: # percorre as linhas do arquivo
            line = line.strip() # separa as palavras de cada linha
            
            # procura as linhas que começam com begin e end definidos acima
            if re.match(begin, line):
                match = True
                continue
            elif re.match(end, line):
                match = False
                continue
            elif match: # quando a linha com o padrão é encontrada
                
                sline = line.split() # separa as palavras 
                if len(sline) == 5: 
                    ltmp = [sline[0] + ' ' + sline[1]] + sline[2:]
                elif len(sline) == 3:
                    ltmp = [''] + sline
                else: 
                    ltmp = sline                    
                    
                if ltmp[0] == 'Observation Type': # se o primeiro elementos for a string 'Observation Type', passa para a próxima linha
                    pass
                elif ltmp[1] == 'Nobs': # idem
                    pass
                else: # atribui os valores para cada coluna do dataframe
                    ltmp[0] = str(ltmp[0])
                    ltmp[1] = int(ltmp[1])
                    ltmp[2] = float(ltmp[2])
                    ltmp[3] = float(ltmp[3])
                                        
                    # escreve a informação sobre a iteração (MITER OMF, MITER OMA etc.)
                    # aqui sempre serão lidas as variáveis surf pres, temp, wind, moist, gps, rad e Jo Global
                    # para cada uma será atribuído o rótulo daquela iteração
                    if i <= 8: # as primeras 8 linhas são referentes ao OMF BEGIN (início do primeiro outer loop)
                        iter_info = 'OMF'
                    elif i > 8 and i <= 17:
                        iter_info = 'OMF (1st INNER LOOP)'
                    elif i > 17 and i <= 26:
                        iter_info = 'OMF (2nd INNER LOOP)'
                    elif i > 26 and i <= 35:
                        iter_info = 'OMA (AFTER 1st OUTER LOOP)'
                    elif i > 35 and i <= 44:
                        iter_info = 'OMA (1st INNER LOOP)'
                    elif i > 44 and i <= 53:
                        iter_info = 'OMA (2nd INNER LOOP)'
                    elif i > 53 and i <= 62:
                        iter_info = 'OMA (AFTER 2nd OUTER LOOP)'
     
                    ltmp.append(iter_info)
                    
                    dfNobs.loc[i] = ltmp
                    
                i += 1
        
        dfNobs = dfNobs.set_index('Observation Type')
        dfNobs.name = str(mname)
            
    return dfNobs

In [3]:
# Nota: os arquivos de log utilizados neste notebook, podem ser encontrados em /scripts/das/carlos.bastarz/SMNA-Dashboard na máquina Itapemirim
bpath = '/Users/carlos/Documents/GitHub/SMNA-Dashboard'

# Função para obter os dataframes em um intervalo de datas para um determinado experimento

def get_df_Nobs(datai, dataf, nexp, mname):

    datai = datetime.strptime(str(datai), '%Y%m%d%H')
    dataf = datetime.strptime(str(dataf), '%Y%m%d%H')
    
    dataifmt=datai.strftime('%Y%m%d%H')
    
    delta = 6
    data = datai

    data = datai + timedelta(hours=delta)

    log_list = {}

    while (data <= dataf):

        datafmt = data.strftime('%Y%m%d%H')
    
        fname = os.path.join(bpath, nexp, str(datafmt), str('gsiStdout_' + str(datafmt) + '.log')) 
    
        log_list[data] = df_Nobs(fname, nexp, mname)
    
        data = data + timedelta(hours=delta)
    
    dftmp = pd.concat(log_list)
    dftmp.index.names = ['Date', dftmp.index.names[1]]
    dftmp = dftmp.reset_index()
    dftmp.name = str(mname)
    
    return dftmp

In [4]:
%%time

# Obtenção dos datasframes para um intervalo de datas

datai = '2023021600'
dataf = '2023031600'

df_dtc = get_df_Nobs(datai, dataf, 'dataout.fullobs_ncep-2023.jgerd', 'df_dtc')
df_bamh_T0 = get_df_Nobs(datai, dataf, 'dataout.fullobs_bamh_T0-2023', 'df_bamh_T0')
df_bamh_T4 = get_df_Nobs(datai, dataf, 'dataout.fullobs_bamh_T4-2023', 'df_bamh_T4')
df_bamh_GT4AT2 = get_df_Nobs(datai, dataf, 'dataout.fullobs_bamh_GT4AT2-2023', 'df_bamh_GT4AT2')
df_dtc_alex = get_df_Nobs(datai, dataf, 'dataout.fullobs_ncep-2023.alex', 'df_dtc_alex')

CPU times: user 52.6 s, sys: 330 ms, total: 52.9 s
Wall time: 52.9 s


In [5]:
df_dtc

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104308,32537.652151,0.312,OMF
1,2023-02-16 06:00:00,temperature,25065,9857.265337,0.393,OMF
2,2023-02-16 06:00:00,wind,127888,61267.072233,0.479,OMF
3,2023-02-16 06:00:00,moisture,8705,2103.832442,0.242,OMF
4,2023-02-16 06:00:00,gps,291665,600962.196931,2.060,OMF
...,...,...,...,...,...,...
5399,2023-03-16 00:00:00,wind,203048,129312.187759,0.637,OMA (AFTER 2nd OUTER LOOP)
5400,2023-03-16 00:00:00,moisture,22219,4948.997007,0.223,OMA (AFTER 2nd OUTER LOOP)
5401,2023-03-16 00:00:00,gps,264466,392890.280946,1.486,OMA (AFTER 2nd OUTER LOOP)
5402,2023-03-16 00:00:00,radiance,183884,56169.185410,0.305,OMA (AFTER 2nd OUTER LOOP)


In [6]:
df_bamh_T0

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104257,33479.040873,0.321,OMF
1,2023-02-16 06:00:00,temperature,25065,9182.537635,0.366,OMF
2,2023-02-16 06:00:00,wind,127924,61756.279429,0.483,OMF
3,2023-02-16 06:00:00,moisture,8705,1898.683053,0.218,OMF
4,2023-02-16 06:00:00,gps,294751,521482.299941,1.769,OMF
...,...,...,...,...,...,...
5399,2023-03-16 00:00:00,wind,203004,127530.106831,0.628,OMA (AFTER 2nd OUTER LOOP)
5400,2023-03-16 00:00:00,moisture,22219,4547.777678,0.205,OMA (AFTER 2nd OUTER LOOP)
5401,2023-03-16 00:00:00,gps,264971,409380.583673,1.545,OMA (AFTER 2nd OUTER LOOP)
5402,2023-03-16 00:00:00,radiance,184436,55546.012353,0.301,OMA (AFTER 2nd OUTER LOOP)


In [7]:
df_bamh_T4

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104272,35096.733688,0.337,OMF
1,2023-02-16 06:00:00,temperature,25066,9503.859322,0.379,OMF
2,2023-02-16 06:00:00,wind,127378,61873.631410,0.486,OMF
3,2023-02-16 06:00:00,moisture,8705,1827.055002,0.210,OMF
4,2023-02-16 06:00:00,gps,294747,523230.572021,1.775,OMF
...,...,...,...,...,...,...
5399,2023-03-16 00:00:00,wind,203274,106677.981278,0.525,OMA (AFTER 2nd OUTER LOOP)
5400,2023-03-16 00:00:00,moisture,22220,2587.152103,0.116,OMA (AFTER 2nd OUTER LOOP)
5401,2023-03-16 00:00:00,gps,268418,336409.760020,1.253,OMA (AFTER 2nd OUTER LOOP)
5402,2023-03-16 00:00:00,radiance,184817,53932.841762,0.292,OMA (AFTER 2nd OUTER LOOP)


In [8]:
df_bamh_GT4AT2

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104294,38100.124937,0.365,OMF
1,2023-02-16 06:00:00,temperature,25066,10703.905918,0.427,OMF
2,2023-02-16 06:00:00,wind,126994,64416.390362,0.507,OMF
3,2023-02-16 06:00:00,moisture,8705,2004.920541,0.230,OMF
4,2023-02-16 06:00:00,gps,293880,553807.345858,1.884,OMF
...,...,...,...,...,...,...
5399,2023-03-16 00:00:00,wind,203352,91804.660711,0.451,OMA (AFTER 2nd OUTER LOOP)
5400,2023-03-16 00:00:00,moisture,22219,1580.502468,0.071,OMA (AFTER 2nd OUTER LOOP)
5401,2023-03-16 00:00:00,gps,269849,278096.752940,1.031,OMA (AFTER 2nd OUTER LOOP)
5402,2023-03-16 00:00:00,radiance,183574,53180.083972,0.290,OMA (AFTER 2nd OUTER LOOP)


In [9]:
df_dtc_alex

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104290,31627.976301,0.303,OMF
1,2023-02-16 06:00:00,temperature,25065,9424.496391,0.376,OMF
2,2023-02-16 06:00:00,wind,128080,61211.409813,0.478,OMF
3,2023-02-16 06:00:00,moisture,8705,2239.077610,0.257,OMF
4,2023-02-16 06:00:00,gps,295128,522537.519588,1.771,OMF
...,...,...,...,...,...,...
5399,2023-03-16 00:00:00,wind,202988,129275.542969,0.637,OMA (AFTER 2nd OUTER LOOP)
5400,2023-03-16 00:00:00,moisture,22219,4934.561534,0.222,OMA (AFTER 2nd OUTER LOOP)
5401,2023-03-16 00:00:00,gps,264686,391207.856391,1.478,OMA (AFTER 2nd OUTER LOOP)
5402,2023-03-16 00:00:00,radiance,182302,56070.153876,0.308,OMA (AFTER 2nd OUTER LOOP)


In [10]:
# Concatenação dos dataframes dos experimentos

dfs = pd.concat([df_dtc, df_bamh_T0, df_bamh_T4, df_bamh_GT4AT2, df_dtc_alex], axis=1, keys=([df_dtc.name, df_bamh_T0.name, df_bamh_T4.name, df_bamh_GT4AT2.name, df_dtc_alex.name]))

In [11]:
dfs

df_dtc                                                  \
                    Date  Observation Type    Nobs             Jo   Jo/n   
0    2023-02-16 06:00:00  surface pressure  104308   32537.652151  0.312   
1    2023-02-16 06:00:00       temperature   25065    9857.265337  0.393   
2    2023-02-16 06:00:00              wind  127888   61267.072233  0.479   
3    2023-02-16 06:00:00          moisture    8705    2103.832442  0.242   
4    2023-02-16 06:00:00               gps  291665  600962.196931  2.060   
...                  ...               ...     ...            ...    ...   
5399 2023-03-16 00:00:00              wind  203048  129312.187759  0.637   
5400 2023-03-16 00:00:00          moisture   22219    4948.997007  0.223   
5401 2023-03-16 00:00:00               gps  264466  392890.280946  1.486   
5402 2023-03-16 00:00:00          radiance  183884   56169.185410  0.305   
5403 2023-03-16 00:00:00         Jo Global  832986  645663.456547  0.775   

                                          df_bamh_T0                    \
                            Iter                Date  Observation Type   
0                            OMF 2023-02-16 06:00:00  surface pressure   
1                            OMF 2023-02-16 06:00:00       temperature   
2                            OMF 2023-02-16 06:00:00              wind   
3                            OMF 2023-02-16 06:00:00          moisture   
4                            OMF 2023-02-16 06:00:00               gps   
...                          ...                 ...               ...   
5399  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00              wind   
5400  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00          moisture   
5401  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00               gps   
5402  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00          radiance   
5403  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00         Jo Global   

                             ... df_bamh_GT4AT2                        \
        Nobs             Jo  ...           Nobs             Jo   Jo/n   
0     104257   33479.040873  ...         104294   38100.124937  0.365   
1      25065    9182.537635  ...          25066   10703.905918  0.427   
2     127924   61756.279429  ...         126994   64416.390362  0.507   
3       8705    1898.683053  ...           8705    2004.920541  0.230   
4     294751  521482.299941  ...         293880  553807.345858  1.884   
...      ...            ...  ...            ...            ...    ...   
5399  203004  127530.106831  ...         203352   91804.660711  0.451   
5400   22219    4547.777678  ...          22219    1580.502468  0.071   
5401  264971  409380.583673  ...         269849  278096.752940  1.031   
5402  184436   55546.012353  ...         183574   53180.083972  0.290   
5403  833987  655463.038472  ...         838398  463291.810295  0.553   

                                         df_dtc_alex                    \
                            Iter                Date  Observation Type   
0                            OMF 2023-02-16 06:00:00  surface pressure   
1                            OMF 2023-02-16 06:00:00       temperature   
2                            OMF 2023-02-16 06:00:00              wind   
3                            OMF 2023-02-16 06:00:00          moisture   
4                            OMF 2023-02-16 06:00:00               gps   
...                          ...                 ...               ...   
5399  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00              wind   
5400  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00          moisture   
5401  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00               gps   
5402  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00          radiance   
5403  OMA (AFTER 2nd OUTER LOOP) 2023-03-16 00:00:00         Jo Global   

                                                                
        Nobs             Jo   Jo/n                        Iter  
0     104290   31627.97

In [12]:
# Escrita do dataframe concatenado em disco no formato CSV

dfs.to_csv('jo_table_series.csv', index=False)

## Exemplos de indexação do dataframe `dfs`

In [13]:
# Escolha de um subdataframe

df_dtc1 = dfs.xs('df_dtc', axis=1)

In [14]:
df_dtc1

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104308,32537.652151,0.312,OMF
1,2023-02-16 06:00:00,temperature,25065,9857.265337,0.393,OMF
2,2023-02-16 06:00:00,wind,127888,61267.072233,0.479,OMF
3,2023-02-16 06:00:00,moisture,8705,2103.832442,0.242,OMF
4,2023-02-16 06:00:00,gps,291665,600962.196931,2.060,OMF
...,...,...,...,...,...,...
5399,2023-03-16 00:00:00,wind,203048,129312.187759,0.637,OMA (AFTER 2nd OUTER LOOP)
5400,2023-03-16 00:00:00,moisture,22219,4948.997007,0.223,OMA (AFTER 2nd OUTER LOOP)
5401,2023-03-16 00:00:00,gps,264466,392890.280946,1.486,OMA (AFTER 2nd OUTER LOOP)
5402,2023-03-16 00:00:00,radiance,183884,56169.185410,0.305,OMA (AFTER 2nd OUTER LOOP)


In [15]:
# Escolha de uma variável

df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].reset_index(drop=True)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104308,32537.652151,0.312,OMF
1,2023-02-16 06:00:00,surface pressure,104308,32537.652151,0.312,OMF (1st INNER LOOP)
2,2023-02-16 06:00:00,surface pressure,104308,19565.932949,0.188,OMF (2nd INNER LOOP)
3,2023-02-16 06:00:00,surface pressure,104493,21515.901229,0.206,OMA (AFTER 1st OUTER LOOP)
4,2023-02-16 06:00:00,surface pressure,104493,21515.901229,0.206,OMA (1st INNER LOOP)
...,...,...,...,...,...,...
765,2023-03-16 00:00:00,surface pressure,101737,19560.846723,0.192,OMF (2nd INNER LOOP)
766,2023-03-16 00:00:00,surface pressure,101842,20747.269002,0.204,OMA (AFTER 1st OUTER LOOP)
767,2023-03-16 00:00:00,surface pressure,101842,20747.269002,0.204,OMA (1st INNER LOOP)
768,2023-03-16 00:00:00,surface pressure,101842,17495.185031,0.172,OMA (2nd INNER LOOP)


In [16]:
# Escolha de um parâmetro

df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].loc[df_dtc1['Iter'] == 'OMF'].reset_index(drop=True)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-16 06:00:00,surface pressure,104308,32537.652151,0.312,OMF
1,2023-02-16 12:00:00,surface pressure,106985,37032.040725,0.346,OMF
2,2023-02-16 18:00:00,surface pressure,104952,33780.264729,0.322,OMF
3,2023-02-17 00:00:00,surface pressure,103821,32335.736629,0.311,OMF
4,2023-02-17 06:00:00,surface pressure,105787,37421.683731,0.354,OMF
...,...,...,...,...,...,...
105,2023-03-15 00:00:00,surface pressure,103366,37381.033459,0.362,OMF
106,2023-03-15 06:00:00,surface pressure,103556,36224.633660,0.350,OMF
107,2023-03-15 12:00:00,surface pressure,106001,36804.312742,0.347,OMF
108,2023-03-15 18:00:00,surface pressure,104646,33684.545327,0.322,OMF


In [17]:
# Escolha de um horário

df_dtc1.loc[df_dtc1['Observation Type'] == 'surface pressure'].loc[df_dtc1['Iter'] == 'OMF'].set_index('Date').at_time(str('00:00:00')).reset_index(drop=False)

,Date,Observation Type,Nobs,Jo,Jo/n,Iter
0,2023-02-17,surface pressure,103821,32335.736629,0.311,OMF
1,2023-02-18,surface pressure,103416,36894.680395,0.357,OMF
2,2023-02-19,surface pressure,102461,40689.066330,0.397,OMF
3,2023-02-20,surface pressure,101196,44363.698481,0.438,OMF
4,2023-02-21,surface pressure,103132,35528.942850,0.344,OMF
5,2023-02-22,surface pressure,105342,37243.800488,0.354,OMF
6,2023-02-23,surface pressure,106545,32733.935103,0.307,OMF
7,2023-02-24,surface pressure,104120,35152.092709,0.338,OMF
8,2023-02-25,surface pressure,103931,34045.808104,0.328,OMF
9,2023-02-26,surface pressure,103663,34116.258323,0.329,OMF
